In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 100
trials =1000
adapt_c = False


size weirdness rn...

In [5]:
a = 0.25
num_params=9

calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/10) )
c1,c2 = calc_c(nobs)

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.72, 5th: 4.11, 10th: 5.15, 30th: 7.77, 50th: 9.91, 70th: 12.55, 90th: 17.24, 95th: 19.68, 99th: 26.07
bonus term 0.0017003668861370503 // variance stats 0.370755068449133 //part1 1.9952623149688797 //part2 0.0008522021758144472
-2.0319368338831025 1.7295854581648236 2.0143348579963734
-2.0353375676553767 1.7295854581648236 2.0109341242240992
---
Quantiles: 1th: 2.48, 5th: 4.05, 10th: 4.95, 30th: 7.26, 50th: 9.38, 70th: 11.79, 90th: 16.12, 95th: 18.72, 99th: 23.50
bonus term 0.0019728733474345204 // variance stats 0.31950021929617084 //part1 1.9952623149688797 //part2 0.0009887789352976835
-1.9520877532867411 -0.8611757742985994 2.1005805144969596
-1.95603349998161 -0.8611757742985994 2.0966347678020902
---
Quantiles: 1th: 2.73, 5th: 4.23, 10th: 5.02, 30th: 7.58, 50th: 9.84, 70th: 12.53, 90th: 16.91, 95th: 19.55, 99th: 25.12
bonus term 0.0020132955446313127 // variance stats 0.3130790639300854 //part1 1.9952623149688797 //part2 0.0010090380244878801
-1.7135108903188434

# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:

data_tuned_c = .8
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.46, 5th: 3.78, 10th: 4.66, 30th: 6.94, 50th: 9.00, 70th: 11.48, 90th: 15.92, 95th: 18.72, 99th: 24.07
bonus term 0.07992065405865918 // variance stats 0.3063405945996496 //part1 3.017088168272582 //part2 0.02648933329131622
-1.6808902148262939 1.3937962752273991 1.8939418899128189
-1.8407315229436123 1.3937962752273991 1.7341005817955004
---
Quantiles: 1th: 2.68, 5th: 4.01, 10th: 4.90, 30th: 7.31, 50th: 9.45, 70th: 12.16, 90th: 16.99, 95th: 19.91, 99th: 25.83
bonus term 0.0733428162874318 // variance stats 0.35635976199136987 //part1 3.017088168272582 //part2 0.024309139208690694
-1.8572373746557034 2.168173698597073 2.037491712059626
-2.003923007230567 2.168173698597073 1.8908060794847628
---
Quantiles: 1th: 2.37, 5th: 3.57, 10th: 4.47, 30th: 6.72, 50th: 8.75, 70th: 11.35, 90th: 15.65, 95th: 18.29, 99th: 24.57
bonus term 0.08088663107909748 // variance stats 0.2979373953816667 //part1 3.017088168272582 //part2 0.026809501932921205
-1.9459090949737705 2.01736830630698

In [8]:

data_tuned_c = .95
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/3) )

nobs = 250
c1,c2 = calc_c(nobs)

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.55, 5th: 3.86, 10th: 4.78, 30th: 7.36, 50th: 9.82, 70th: 12.57, 90th: 18.06, 95th: 21.37, 99th: 30.08
bonus term 0.2661976142874326 // variance stats 0.3063756157534467 //part1 6.299605249474365 //part2 0.04225623729512956
-1.601483040982219 1.865335438691927 1.9702313067343453
-2.133878269557084 1.865335438691927 1.43783607815948
---
Quantiles: 1th: 2.47, 5th: 3.80, 10th: 4.73, 30th: 7.14, 50th: 9.26, 70th: 11.83, 90th: 16.46, 95th: 19.11, 99th: 24.41
bonus term 0.196410232973783 // variance stats 0.37556187003198455 //part1 6.299605249474365 //part2 0.03117818104399023
-1.632045625674612 2.33115345383471 2.2582628138843757
-2.024866091622178 2.33115345383471 1.8654423479368094
---
Quantiles: 1th: 2.51, 5th: 3.71, 10th: 4.59, 30th: 6.97, 50th: 9.10, 70th: 11.80, 90th: 16.86, 95th: 19.90, 99th: 27.62
bonus term 0.3029668508868243 // variance stats 0.24913151186309082 //part1 6.299605249474365 //part2 0.04809298978092375
-1.5819713487553206 0.019267764098071877 1.95716

In [9]:

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/250,nobs**(1/3) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.58, 5th: 3.92, 10th: 4.88, 30th: 7.55, 50th: 9.86, 70th: 12.91, 90th: 18.85, 95th: 22.50, 99th: 30.02
bonus term 0.07716093524417057 // variance stats 0.38456758705545097 //part1 6.299605249474365 //part2 0.012248534977744651
-1.8120364412129348 1.8801668383349197 2.384261509743154
-1.9663583117012762 1.8801668383349197 2.2299396392548125
---
Quantiles: 1th: 2.51, 5th: 3.93, 10th: 4.86, 30th: 7.20, 50th: 9.48, 70th: 11.93, 90th: 16.68, 95th: 19.59, 99th: 26.05
bonus term 0.10019289287362804 // variance stats 0.26114965810566687 //part1 6.299605249474365 //part2 0.015904630354733426
-2.0072300868335313 0.7019470717623796 2.0661029761744847
-2.2076158725807877 0.7019470717623796 1.8657171904272283
---
Quantiles: 1th: 2.83, 5th: 4.16, 10th: 5.24, 30th: 7.85, 50th: 10.19, 70th: 13.17, 90th: 18.50, 95th: 21.75, 99th: 28.38
bonus term 0.09437685965796418 // variance stats 0.3077790543738304 //part1 6.299605249474365 //part2 0.014981392630250747
-1.819918421633809 2.32010362

In [10]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.62, 5th: 4.14, 10th: 5.02, 30th: 7.65, 50th: 10.16, 70th: 13.27, 90th: 19.13, 95th: 23.38, 99th: 31.97
bonus term 0.1558220698256124 // variance stats 0.2455755547392915 //part1 1.736976732219687 //part2 0.08970878362111794
-1.5799469557270254 0.7097699841959565 1.916132233933457
-1.89159109537825 0.7097699841959565 1.6044880942822324
---
Quantiles: 1th: 2.53, 5th: 3.95, 10th: 4.85, 30th: 7.45, 50th: 9.69, 70th: 12.47, 90th: 17.37, 95th: 20.22, 99th: 25.69
bonus term 0.14011508095333375 // variance stats 0.25042825472150826 //part1 1.736976732219687 //part2 0.08066606670906888
-1.6350078052834978 1.622197186976505 1.8855423742804758
-1.9152379671901654 1.622197186976505 1.6053122123738082
---
Quantiles: 1th: 2.64, 5th: 3.93, 10th: 4.85, 30th: 7.23, 50th: 9.36, 70th: 12.08, 90th: 16.56, 95th: 18.96, 99th: 24.40
bonus term 0.09225549579812863 // variance stats 0.37342264652089296 //part1 1.736976732219687 //part2 0.05311268371467193
-1.8960084042576073 1.849687935845670

In [11]:

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/10) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.40, 5th: 3.87, 10th: 4.84, 30th: 7.27, 50th: 9.64, 70th: 12.73, 90th: 17.97, 95th: 21.05, 99th: 27.55
bonus term 0.05880595148145308 // variance stats 0.3243687040235078 //part1 1.736976732219687 //part2 0.03385534785276301
-1.888674049949607 2.8739490536435195 2.238101864075285
-2.0062859529125134 2.8739490536435195 2.1204899611123786
---
Quantiles: 1th: 3.12, 5th: 4.45, 10th: 5.40, 30th: 8.19, 50th: 10.63, 70th: 13.77, 90th: 20.31, 95th: 23.98, 99th: 31.86
bonus term 0.06295341314990888 // variance stats 0.3415048131469331 //part1 1.736976732219687 //part2 0.03624309524829417
-1.9210462414534526 2.1978744958778362 2.1654388978993167
-2.04695306775327 2.1978744958778362 2.039532071599499
---
Quantiles: 1th: 2.32, 5th: 3.45, 10th: 4.30, 30th: 6.38, 50th: 8.33, 70th: 10.51, 90th: 14.62, 95th: 16.98, 99th: 21.69
bonus term 0.046301336375961184 // variance stats 0.3376236842657972 //part1 1.736976732219687 //part2 0.02665627899159742
-1.859773927241934 1.841397137803678 

# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.40, 5th: 3.90, 10th: 4.78, 30th: 7.18, 50th: 9.35, 70th: 12.03, 90th: 16.91, 95th: 19.79, 99th: 26.71
bonus term 0.04302311879871437 // variance stats 0.42099661274894384 //part1 1.736976732219687 //part2 0.024768966676793083
-1.8987215362605503 -0.4583272967326806 1.9615369919776295
-1.9847677738579792 -0.4583272967326806 1.8754907543802006
---
Quantiles: 1th: 2.45, 5th: 3.92, 10th: 4.98, 30th: 7.48, 50th: 9.79, 70th: 12.49, 90th: 17.84, 95th: 21.20, 99th: 27.07
bonus term 0.045920821011012074 // variance stats 0.4154573133561383 //part1 1.736976732219687 //part2 0.02643721136801282
-1.8186353335397905 0.3598137651023631 2.027373834626226
-1.9104769755618145 0.3598137651023631 1.9355321926042015
---
Quantiles: 1th: 2.50, 5th: 3.83, 10th: 4.75, 30th: 7.18, 50th: 9.44, 70th: 12.11, 90th: 17.08, 95th: 20.16, 99th: 26.24
bonus term 0.051199158133019694 // variance stats 0.3556323653759412 //part1 1.736976732219687 //part2 0.029476018407910483
-1.6801918502770639 0.029660

In [14]:

nobs = 500
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.61, 5th: 3.90, 10th: 4.71, 30th: 6.86, 50th: 8.89, 70th: 11.21, 90th: 15.29, 95th: 17.46, 99th: 22.97
bonus term 0.00215338773373887 // variance stats 0.386177635407579 //part1 1.8616455666360694 //part2 0.0011567119823081948
-2.0008838006105467 -1.804935437876384 1.854889738941538
-2.005190576078024 -1.804935437876384 1.85058296347406
---
Quantiles: 1th: 2.53, 5th: 3.88, 10th: 4.78, 30th: 7.13, 50th: 9.20, 70th: 11.73, 90th: 16.23, 95th: 18.64, 99th: 23.44
bonus term 0.0026524627147969638 // variance stats 0.31343211174144525 //part1 1.8616455666360694 //part2 0.0014247946882767132
-1.7374984512827947 0.6473315622601927 2.0379336831987223
-1.7428033767123887 0.6473315622601927 2.032628757769128
---
Quantiles: 1th: 2.66, 5th: 4.05, 10th: 5.00, 30th: 7.50, 50th: 9.58, 70th: 12.20, 90th: 17.06, 95th: 19.61, 99th: 26.31
bonus term 0.0021635839933684523 // variance stats 0.3843555996780828 //part1 1.8616455666360694 //part2 0.0011621889967368898
-1.9997513252838515 1.9222

In [15]:
nobs = 100
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.41, 5th: 3.75, 10th: 4.63, 30th: 7.10, 50th: 9.39, 70th: 12.14, 90th: 17.41, 95th: 20.70, 99th: 28.25
bonus term 0.04794041547344633 // variance stats 0.5582243548899309 //part1 1.5848931924611136 //part2 0.030248357240402857
-1.6771285933861813 -0.547580286059411 1.9278990496837412
-1.773009424333074 -0.547580286059411 1.8320182187368483
---
Quantiles: 1th: 2.11, 5th: 3.69, 10th: 4.52, 30th: 7.13, 50th: 9.68, 70th: 13.27, 90th: 20.19, 95th: 24.94, 99th: 35.58
bonus term 0.05838998919667126 // variance stats 0.527799868753819 //part1 1.5848931924611136 //part2 0.036841592527758864
-1.8369293180027353 -0.767958073738552 1.7621490040610466
-1.9537092963960778 -0.767958073738552 1.645369025667704
---
Quantiles: 1th: 2.32, 5th: 3.58, 10th: 4.52, 30th: 6.95, 50th: 9.44, 70th: 12.37, 90th: 18.17, 95th: 21.40, 99th: 29.26
bonus term 0.08548288903770043 // variance stats 0.3187681155140616 //part1 1.5848931924611136 //part2 0.05393605666572248
-1.993547651938668 0.22596872869

# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.97 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.04 & 0.04 & 0.02 & 0.02 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.95, 0.01, 0.04]), array([0.95, 0.01, 0.04]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), -1.1526542304913558, 5.618623013502887, 5.580559738669481)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.70 & 0.78 & 0.68 & 0.96 & 0.96 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.30 & 0.22 & 0.32 & 0.03 & 0.03 & 0.03   \\
\hline
\end{tabular}
(array([0.7, 0. , 0.3]), array([0.78, 0.  , 0.22]), array([0.68, 0.  , 0.32]), array([0.96, 0.01, 0.03]), array([0.96, 0.01, 0.03]), array([0.97, 0.  , 0.03]), -9.889474035117619, 6.4703231111739745, 6.714136150645025)


# evidence of power

In [20]:
num_params= 9
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.77 & 0.49 & 0.53 & 0.52   \\
Model 1 & 0.20 & 0.20 & 0.23 & 0.51 & 0.47 & 0.48   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.8, 0.2, 0. ]), array([0.8, 0.2, 0. ]), array([0.77, 0.23, 0.  ]), array([0.49, 0.51, 0.  ]), array([0.53, 0.47, 0.  ]), array([0.52, 0.48, 0.  ]), 7.28008528029158, 5.100197521079789, 5.418091845391691)


In [21]:
nobs = 500
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.78 & 0.78 & 0.80 & 0.40 & 0.47 & 0.43   \\
Model 1 & 0.22 & 0.22 & 0.20 & 0.60 & 0.53 & 0.57   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.78, 0.22, 0.  ]), array([0.78, 0.22, 0.  ]), array([0.8, 0.2, 0. ]), array([0.4, 0.6, 0. ]), array([0.47, 0.53, 0.  ]), array([0.43, 0.57, 0.  ]), 7.411004817710161, 5.083485643444695, 5.382844448368343)


In [ ]:

nobs = 100
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
num_params= 19
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
num_params= 4
nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

# evidence of power 2

In [ ]:
num_params= 9
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 500
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 100
c1,c2 = calc_c(nobs)



a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
num_params= 19
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
num_params= 4
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)